## Question C: Location-Based Promotion Strategy
### Data and Methodology
- **Data Sources:** Geographic sales data, regional demographics, and market trends.
- **Methodology:** Geographic Information System (GIS) analysis and clustering techniques to identify key locations for promotions.

### Import Necessary Libraries

In [17]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd
import boto3

## Information about `BANGLADESH: Districts and Cities`

In [2]:
url = "https://www.citypopulation.de/en/bangladesh/cities/"
response = requests.get(url)
html_content = response.text

### Scraping the Data using `Beautiful Soup`

In [36]:
# Assuming html_content contains the HTML data

soup = BeautifulSoup(html_content, "html.parser")

# Find the table containing the data
table = soup.find("table")

# Extract rows from the table
rows = table.find_all("tr")

# Initialize a set to store unique city names
unique_cities = set()

# Initialize a list to store data
places = []

# Loop through rows to extract data
for row in rows[1:]:  # Skip the header row
    cells = row.find_all("td")
    if len(cells) >= 7:  # Ensure that row has enough cells
        name = cells[1].text.strip()
        # Check if the city is unique
        if name not in unique_cities:
            area = float(cells[5].text.strip().replace(',', ''))  # Convert to float, remove commas
            population = int(cells[8].text.strip().replace(',', ''))  # Convert to int, remove commas
            density = population / area  # Calculate density
            places.append((name, density))
            unique_cities.add(name)  # Add city to set of unique cities

# Sort places by density in descending order
places.sort(key=lambda x: x[1], reverse=True)

# Display top 10 highest density places
print("Top 20 highest density places:")
for i, (name, density) in enumerate(places[:20], 1):
    print(f"{i}. {name}: {density:.2f} people per square unit")


Top 20 highest density places:
1. Narayanganj: 4494.27 people per square unit
2. Narsingdi: 2012.95 people per square unit
3. Gazipur: 1964.63 people per square unit
4. Dhaka: 1847.67 people per square unit
5. Cumilla: 1780.87 people per square unit
6. Brahmanbaria: 1570.02 people per square unit
7. Chandpur: 1528.17 people per square unit
8. Feni: 1511.25 people per square unit
9. Munshiganj: 1496.46 people per square unit
10. Sirajganj: 1340.89 people per square unit
11. Kushtia: 1257.71 people per square unit
12. Lakshmipur: 1248.45 people per square unit
13. Nilphamari: 1233.03 people per square unit
14. Bogra: 1220.87 people per square unit
15. Gaibandha: 1168.64 people per square unit
16. Jamalpur: 1127.57 people per square unit
17. Kishoreganj: 1126.33 people per square unit
18. Pabna: 1104.66 people per square unit
19. Jessore: 1103.33 people per square unit
20. Tangail: 1098.15 people per square unit


### The Co-ordinates are collected from Github

In [38]:
# Load GeoJSON file containing district boundaries
geojson_path = "bangladesh_geojson_adm2_64_districts_zillas.json"  # Provide the path to your GeoJSON file
districts_gdf = gpd.read_file(geojson_path)

In [39]:
# Extract district names and coordinates
district_coordinates = {}

In [40]:
print(districts_gdf.head(20))

    Shape_Leng  Shape_Area       ADM2_EN ADM2_PCODE    ADM2_REF ADM2ALT1EN  \
0    12.929816    0.319709      Bagerhat     BD4001        None       None   
1     5.358126    0.401359     Bandarban     BD2003        None       None   
2     4.167970    0.117801       Barguna     BD1004        None       None   
3     7.713625    0.195228       Barisal     BD1006        None       None   
4    10.091155    0.170812         Bhola     BD1009        None       None   
5     4.289236    0.260098        Bogura     BD5010        None       None   
6     3.598531    0.170544  Brahmanbaria     BD2012        None       None   
7     4.332182    0.129682      Chandpur     BD2013        None       None   
8     8.259337    0.390465    Chattogram     BD2015        None       None   
9     2.328542    0.103078     Chuadanga     BD4018        None       None   
10    3.833726    0.272856       Comilla     BD2019        None       None   
11    6.249755    0.189378   Cox's Bazar     BD2022  Coxs Bazar 

In [41]:
for index, row in districts_gdf.iterrows():
    district_name = row["ADM2_EN"]  # Adjust column name accordingly
    coordinates = list(row["geometry"].centroid.coords)[0]  # Centroid coordinates
    district_coordinates[district_name] = coordinates

In [42]:
# Assuming district_coordinates is a dictionary containing district names as keys and their corresponding coordinates as values
district_names = [
    "Narayanganj",
    "Narsingdi",
    "Gazipur",
    "Dhaka",
    "Brahmanbaria",
    "Chandpur",
    "Feni",
    "Munshiganj",
    "Sirajganj",
    "Kushtia",
]

for district_name in district_names:
    coordinates = district_coordinates.get(district_name)
    if coordinates is not None:
        print(f"District: {district_name}, Coordinates: {coordinates}")
    else:
        print(f"No coordinates found for {district_name}")


District: Narayanganj, Coordinates: (90.57765132100197, 23.72253372193042)
District: Narsingdi, Coordinates: (90.77322783176406, 24.0021468812996)
District: Gazipur, Coordinates: (90.44497358349848, 24.098179572025913)
District: Dhaka, Coordinates: (90.25063549805675, 23.787080416912993)
District: Brahmanbaria, Coordinates: (91.08375058470862, 23.953438255117913)
District: Chandpur, Coordinates: (90.77217805161908, 23.27440454466741)
District: Feni, Coordinates: (91.41211314831683, 22.99910848428237)
District: Munshiganj, Coordinates: (90.41694537036828, 23.52605097621976)
District: Sirajganj, Coordinates: (89.59979755857451, 24.39116893182422)
District: Kushtia, Coordinates: (89.02222249234792, 23.922852701805148)


In [43]:
# Assuming district_coordinates is a dictionary containing district names as keys and their corresponding coordinates as values
district_names = [
    "Narayanganj",
    "Narsingdi",
    "Gazipur",
    "Dhaka",
    "Brahmanbaria",
    "Chandpur",
    "Feni",
    "Munshiganj",
    "Sirajganj",
    "Kushtia",
]

# Collect data into a list of dictionaries
data = []
for district_name in district_names:
    coordinates = district_coordinates.get(district_name)
    if coordinates is not None:
        data.append({'District': district_name, 'Coordinates': coordinates})
    else:
        data.append({'District': district_name, 'Coordinates': None})

# Create DataFrame
df = pd.DataFrame(data)
print(df)


       District                              Coordinates
0   Narayanganj   (90.57765132100197, 23.72253372193042)
1     Narsingdi    (90.77322783176406, 24.0021468812996)
2       Gazipur  (90.44497358349848, 24.098179572025913)
3         Dhaka  (90.25063549805675, 23.787080416912993)
4  Brahmanbaria  (91.08375058470862, 23.953438255117913)
5      Chandpur   (90.77217805161908, 23.27440454466741)
6          Feni   (91.41211314831683, 22.99910848428237)
7    Munshiganj   (90.41694537036828, 23.52605097621976)
8     Sirajganj   (89.59979755857451, 24.39116893182422)
9       Kushtia  (89.02222249234792, 23.922852701805148)


In [44]:
data = [
    {'District': 'Narayanganj', 'Coordinate': (90.57765132100197, 23.72253372193042)},
    {'District': 'Narsingdi', 'Coordinate': (90.77322783176406, 24.0021468812996)},
    {'District': 'Gazipur', 'Coordinate': (90.44497358349848, 24.098179572025913)},
    {'District': 'Dhaka', 'Coordinate': (90.25063549805675, 23.787080416912993)},
    {'District': 'Brahmanbaria', 'Coordinate': (91.08375058470862, 23.953438255117913)},
    {'District': 'Chandpur', 'Coordinate': (90.77217805161908, 23.27440454466741)},
    {'District': 'Feni', 'Coordinate': (91.41211314831683, 22.99910848428237)},
    {'District': 'Munshiganj', 'Coordinate': (90.41694537036828, 23.52605097621976)},
    {'District': 'Sirajganj', 'Coordinate': (89.59979755857451, 24.39116893182422)},
    {'District': 'Kushtia', 'Coordinate': (89.02222249234792, 23.922852701805148)}
]


In [45]:
data

[{'District': 'Narayanganj',
  'Coordinate': (90.57765132100197, 23.72253372193042)},
 {'District': 'Narsingdi',
  'Coordinate': (90.77322783176406, 24.0021468812996)},
 {'District': 'Gazipur',
  'Coordinate': (90.44497358349848, 24.098179572025913)},
 {'District': 'Dhaka', 'Coordinate': (90.25063549805675, 23.787080416912993)},
 {'District': 'Brahmanbaria',
  'Coordinate': (91.08375058470862, 23.953438255117913)},
 {'District': 'Chandpur',
  'Coordinate': (90.77217805161908, 23.27440454466741)},
 {'District': 'Feni', 'Coordinate': (91.41211314831683, 22.99910848428237)},
 {'District': 'Munshiganj',
  'Coordinate': (90.41694537036828, 23.52605097621976)},
 {'District': 'Sirajganj',
  'Coordinate': (89.59979755857451, 24.39116893182422)},
 {'District': 'Kushtia',
  'Coordinate': (89.02222249234792, 23.922852701805148)}]

In [46]:
# Create DataFrame
df = pd.DataFrame(data)

# Remove rows with None coordinates
df = df.dropna(subset=['Coordinate'])

# Initialize a Folium map centered around an average location
map_center = [23.7, 90.35]  # Adjust center if needed
map = folium.Map(location=map_center, zoom_start=7)

# Add markers and labels for each district
for index, row in df.iterrows():
    # Add a standard marker
    folium.Marker(
        location=[row['Coordinate'][1], row['Coordinate'][0]],  # Latitude, Longitude
        popup=row['District'],
        icon=folium.Icon(color='red')
    ).add_to(map)
    
    # Add a label with DivIcon
    folium.Marker(
        location=[row['Coordinate'][1], row['Coordinate'][0]],  # Latitude, Longitude
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12pt; color: red; font-weight: bold;">{row["District"]}</div>'
        )
    ).add_to(map)

map

References:
* 1: https://www.citypopulation.de/en/bangladesh/cities/
* 2: https://github.com/ahnaf-tahmid-chowdhury/Choropleth-Bangladesh/blob/master/bangladesh_geojson_adm2_64_districts_zillas.json